In [50]:
#Finalizing the coordinate match module
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from pandas import DataFrame
from astropy import units as u
from astropy.coordinates import SkyCoord
import csv 
import pandas as pd
from astropy.coordinates import match_coordinates_sky
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [51]:
def match_stars(filename, filename2, sep):
    df = pd.read_csv(filename)
    RA = df["RA"]
    DEC = df["DEC"]
    data_rsgRADEC = pd.concat([RA,DEC], axis=1)

    df2 = pd.read_csv(filename2)
    RAstar = df2["RA"]
    DECstar = df2["DEC"]
    data2_rsgRADEC = pd.concat([RAstar,DECstar], axis=1)
    
    rsg_coord = data_rsgRADEC.values.tolist()
    c1 = SkyCoord(rsg_coord, frame='icrs', unit= (u.hourangle, u.deg))
    
    rsg2_coord = data2_rsgRADEC.values.tolist()
    c2 = SkyCoord(rsg2_coord, frame='icrs', unit= (u.hourangle, u.deg))
    
    max_sep = sep * u.arcsec
    idx, d2d, d3d = c2.match_to_catalog_sky(c1)
    sep_constraint = d2d < max_sep
    
    c2_matches = c2[sep_constraint]
    c1_matches = c1[idx[sep_constraint]]

    idx_matches = idx[sep_constraint]
    RSG_match = df.iloc[idx_matches]
    
    angle_sep = d2d[sep_constraint]
    angle_sep_df = DataFrame(angle_sep, columns=['sep'])
    data_match = RSG_match.join(angle_sep_df.set_index(RSG_match.index), on=RSG_match.index)

    distance = d3d[sep_constraint]
    distance_df = DataFrame(distance, columns=['distance'])
    final_match = data_match.join(distance_df.set_index(RSG_match.index), on=RSG_match.index)
        
    return final_match

In [52]:
matched = match_stars('RSG1.csv','RSGpaper1.csv', 2.0)

In [53]:
def hrd(filename, matches, pointsize,pointsize2):
    #matching datasets HRD values
    data = pd.read_csv(filename)
    R =  0.50
    reddening = 0.25
    distance_modulus = 5*np.log10(R * (10**6))-5
    K = data["K"]
    JK = data["J-K"]
    JK_bb = (JK + 0.11) / 0.972
    K_bb = K + 0.044
    JK_bb0 = (JK_bb) - 0.54* reddening
    K_bb0 = (K_bb) - 0.367*reddening
    M_K = K_bb0 - distance_modulus
    temp = 5643.5 - (1807.1 * (JK_bb0))
    BC_K = 5.567 - (7.5686 * (10**(-4))) * temp
    M_bol = BC_K + M_K
    luminosity = (M_bol - 4.75) / -2.5
    T = np.log10(temp)
    L = luminosity
    
    #matching datasets HRD values
    K_match = matches["K"]
    JK_match = matches["J-K"]
    JK_bb_match = (JK_match + 0.11) / 0.972
    K_bb_match = K_match + 0.044
    JK_bb0_match = (JK_bb_match) - 0.54* reddening
    K_bb0_match = (K_bb_match) - 0.367*reddening
    M_K_match = K_bb0_match - distance_modulus
    temp_match = 5643.5 - (1807.1 * (JK_bb0_match))
    BC_K_match = 5.567 - (7.5686 * (10**(-4))) * temp_match
    M_bol_match = BC_K_match + M_K_match
    luminosity_match = (M_bol_match - 4.75) / -2.5
    T_match = np.log10(temp_match)
    L_match = luminosity_match
    
    #making the plot
    fig, ax = plt.subplots(1,1)       
    fig.set_size_inches(13,8)
    ax.set_xlim(3.5,3.65)
    ax.set_ylim(3.9,5.5)
    ax.invert_xaxis()
    ax.set_xlabel("log(temperature)", fontsize=30)
    ax.set_ylabel("log(luminosity)", fontsize=30)
    plt.title('HRD of NGC6822 RSG Sample Matches', fontsize=30)

    ax.plot(T, L,color= "pink", marker="o",linestyle="None",markersize=pointsize, alpha=0.5);
    ax.plot(T_match,L_match,color="blue",marker="o",linestyle="None",markersize=pointsize2);

In [54]:
def interactive_hrd(filename, matches, pointsize, pointsize2):
    interact(hrd, filename = fixed(filename),matches = fixed(matches), pointsize=(0, 10, 1),pointsize2=(0, 10, 1));   

In [55]:
interactive_hrd('RSG1.csv', matched, 6, 4)

interactive(children=(IntSlider(value=5, description='pointsize', max=10), IntSlider(value=5, description='poi…